In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Imports

In [2]:
import torch
import torch.nn.functional as F
from tqdm import tqdm
from torchvision import transforms
from PIL import Image
import numpy as np
import os
import timeit
import torch.nn as nn
from torch.autograd import Function
import csv
from os.path import splitext
from os import listdir
from glob import glob
from torch.utils.data import Dataset
import logging
from torchvision.io import read_image
import torchvision
from torchvision.transforms import functional as F
from torchvision.transforms.functional import resize
import torch.nn.functional as F
from __future__ import absolute_import
import torchvision.models as models
from torchvision.ops import DeformConv2d
import sys
from torch import optim
from torch.utils.data import DataLoader

# Metrics

In [3]:
ALPHA = 0.8
GAMMA = 2

class Dice(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(Dice, self).__init__()

    def forward(self, inputs, targets, eps = 0.0001):

        inputs = F.sigmoid(inputs)
        inputs = (inputs>0.5).int()

        #flatten label and prediction tensors
        intersection = torch.sum(inputs * targets, dim=(1,2,3))
        total = torch.sum(inputs + targets, dim=(1,2,3))
        dice = (2*intersection + eps)/(total + eps)

        return torch.mean(dice)

class IoU(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(IoU, self).__init__()

    def forward(self, inputs, targets, eps = 0.0001):

        inputs = F.sigmoid(inputs)
        inputs = (inputs>0.5).int()

        #flatten label and prediction tensors
        intersection = torch.sum(inputs * targets, dim=(1,2,3))
        total = torch.sum(inputs + targets, dim=(1,2,3))
        union = total - intersection

        IoU = (intersection + eps)/(union + eps)

        return torch.mean(IoU)


dice_coeff = Dice()
jaccard_index = IoU()

def eval_dice_IoU_binary(net, loader, device, test_counter, save_dir, save=True):
    """Evaluation without the densecrf with the dice coefficient"""
    net.eval()
    n_val = len(loader)  # the number of batch
    dice = []
    IoU = []
    Inference_time = []

    try:
       os.makedirs(save_dir)
    except OSError:
       pass

    with tqdm(total=n_val, desc='Validation round', unit='batch', leave=False) as pbar:
        for batch in loader:
            imgs, true_masks, name = batch['image'], batch['mask'], batch['name']

            start = timeit.default_timer()

            with torch.no_grad():
                mask_pred = net(imgs)

            stop = timeit.default_timer()
            Inference_time.append(stop - start)



            if net.n_classes > 1:
                raise Exception("Not implemented for multi-class segmentation")
            else:

                val_Dice = dice_coeff(mask_pred, true_masks).item()
                val_IoU = jaccard_index(mask_pred, true_masks).item()

                dice.append(val_Dice)
                IoU.append(val_IoU)

                pbar.set_postfix(**{'val_Dice (batch)': val_Dice})
                pbar.set_postfix(**{'val_IoU (batch)': val_IoU})

            if save:
                pred = torch.sigmoid(mask_pred)
                pred = (pred > 0.5).float()
                probs = pred.squeeze(0)

                tf = transforms.Compose(
                  [
                   transforms.ToPILImage(),
                   transforms.ToTensor()
                  ]
                  )

                probs = tf(probs.cpu())
                full_mask = Image.fromarray(probs.squeeze().cpu().numpy()*255).convert('RGB')
                full_mask = full_mask.save(save_dir+ str(name[0]) + '_' + str(test_counter) + '.png')

            pbar.update()

    net.train()
    return sum(dice)/n_val, np.std(dice), sum(IoU)/n_val, np.std(IoU), min(dice), min(IoU), max(dice), max(IoU), sum(Inference_time)/n_val

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        inputs = F.sigmoid(inputs)

        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')

        intersection = torch.sum(inputs * targets, dim=(1,2,3))
        total = torch.sum(inputs + targets, dim=(1,2,3))
        dice = (2*intersection + smooth)/(total + smooth)

        print(f'Train Dice: {dice}')
        print(f'Train BCE: {BCE}')
        Dice_BCE = 0.8*BCE - 0.2*torch.log(torch.mean(dice))

        return Dice_BCE


def save_metrics(values, name):

    fields = ['ave(dice)', 'std(dice)', 'ave(IoU)', 'std(IoU)', 'min(dice)', 'min(IoU)', 'max(dice)', 'max(IoU)', 'inference_time', 'epoch_loss']

    with open(name, 'w') as f:

        # using csv.writer method from CSV package
        write = csv.writer(f)
        write.writerow(fields)
        write.writerows(values)

# Transforms

In [4]:

def pad_if_smaller(img, size, fill=0):
    ow, oh = img.size
    min_size = min(oh, ow)
    if min_size < size:
        padh = size - oh if oh < size else 0
        padw = size - ow if ow < size else 0
        img = F.pad(img, (0, 0, padw, padh), fill=fill)
    return img



class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target




class Resize(object):
    def __init__(self, size):
        if isinstance(size, tuple):
            self.size = size
        else:
            self.size = (size, size)

    def __call__(self, image, target):
        image = resize(image, self.size)
        target = resize(target, self.size, interpolation=torchvision.transforms.InterpolationMode.NEAREST)
        return image, target


class ToTensor(object):
    def __call__(self, image, target):
        image = F.to_tensor(image)
        target = F.to_tensor(target.astype(float)).long()
        return image, target


class Normalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, image, target):
        image = F.normalize(image, mean=self.mean, std=self.std)
        return image, target




# Dataset

In [5]:
class BasicDataset(Dataset):
    def __init__(self, imgs_dir, masks_dir, size = (512, 512),
                 device = 'cuda' if torch.cuda.is_available() else 'cpu',
                 scale=1, mask_suffix=''):

        self.imgs_dir = imgs_dir
        self.masks_dir = masks_dir
        self.size = size
        self.device = device
        self.scale = scale
        assert 0 < scale <= 1, 'Scale must be between 0 and 1'
        self.mask_suffix = mask_suffix



        self.ids = [splitext(file)[0] for file in listdir(imgs_dir)
                    if not file.startswith('.')]

        logging.info(f'Creating dataset with {len(self.ids)} examples')


        self.transforms = Compose([Resize(self.size)])


        self.transforms_necessary = Compose([Resize(self.size)])

        print("Here inside dataset")

    def __len__(self):
        return len(self.ids)


    def __getitem__(self, i):
        idx = self.ids[i]

        mask_file = glob(self.masks_dir + '/' + idx + '.*')
        img_file = glob(self.imgs_dir + '/' + idx + '.*')


        assert len(mask_file) == 1, \
            f'Either no mask or multiple masks found for the ID {idx}: {mask_file}'
        assert len(img_file) == 1, \
            f'Either no image or multiple images found for the ID {idx}: {img_file}'


        img = (read_image(img_file[0])/255).type(torch.FloatTensor)
        mask = (read_image(mask_file[0])/255).type(torch.FloatTensor)

        img = img.to(self.device)
        mask = mask.to(self.device)


        assert img.size(dim = 1)== mask.size(dim = 1), \
            f'Image and mask {idx} should be the same size, but are {img.size} and {mask.size}'

        if mask.size(dim = (0)) > 1:
            mask = mask[0,:,:].unsqueeze(0)


        img, mask = self.transforms_necessary(img, mask)


        return {
            'image': img.type(torch.cuda.FloatTensor),#cuda
            'mask': mask.type(torch.cuda.FloatTensor),#cuda
            'name': str(self.ids[i])
        }

# Model

In [6]:
#VGG16 Encoder block
class VGG16_Separate(nn.Module):
    def __init__(self):
        super(VGG16_Separate,self).__init__()

        vgg_model = models.vgg16(pretrained=True)
        self.Conv1 = nn.Sequential(*list(vgg_model.features.children())[0:4])
        self.Conv2 = nn.Sequential(*list(vgg_model.features.children())[4:9])
        self.Conv3 = nn.Sequential(*list(vgg_model.features.children())[9:16])
        self.Conv4 = nn.Sequential(*list(vgg_model.features.children())[16:23])
        self.Conv5 = nn.Sequential(*list(vgg_model.features.children())[23:30])

    def forward(self,x):

        out1 = self.Conv1(x)
        out2 = self.Conv2(out1)
        out3 = self.Conv3(out2)
        out4 = self.Conv4(out3)
        out5 = self.Conv5(out4)

        return out1, out2, out3, out4, out5


#Global average pooling
class PoolUp(nn.Module):
      def __init__(self, input_channels, pool_kernel_size, reduced_channels):
          super().__init__()

          self.pool = nn.AvgPool2d(kernel_size=pool_kernel_size, stride = pool_kernel_size)
          self.up = nn.Upsample(scale_factor=pool_kernel_size)

      def forward(self,x):

          y = self.pool(x)
          y = self.up(y)

          return y


#Pyramid View Fusion
#Uses global average pooling
class PVF(nn.Module):
    def __init__(self, input_channels, pool_sizes, dim):
        super().__init__()

        reduced_channels = input_channels//8
        output_channels = input_channels//2

        self.conv = nn.Conv2d(input_channels, output_channels, kernel_size=1, padding=0)

        self.PoolUp1 = PoolUp(output_channels, pool_sizes[0], reduced_channels)
        self.PoolUp2 = nn.AvgPool2d(kernel_size=pool_sizes[1], padding = (pool_sizes[1])//2, stride=1)
        self.PoolUp3 = nn.AvgPool2d(kernel_size=pool_sizes[2], padding = (pool_sizes[2])//2, stride=1)
        self.PoolUp4 = nn.AvgPool2d(kernel_size=pool_sizes[3], padding = (pool_sizes[3])//2, stride=1)

        self.conv1 = nn.Conv2d(output_channels, reduced_channels, kernel_size=1, padding=0)
        self.conv2 = nn.Conv2d(output_channels, reduced_channels, kernel_size=1, padding=0)
        self.conv3 = nn.Conv2d(output_channels, reduced_channels, kernel_size=1, padding=0)
        self.conv4 = nn.Conv2d(output_channels, reduced_channels, kernel_size=1, padding=0)

        self.fc = nn.Conv2d(input_channels, input_channels, kernel_size=3, padding=1, groups=input_channels)
        self.norm = nn.LayerNorm([dim,dim])


    def forward(self,x):


        y = self.conv(x)

        glob1 = self.conv1(self.PoolUp1(y))
        glob2 = self.conv2(self.PoolUp2(y))
        glob3 = self.conv3(self.PoolUp3(y))
        glob4 = self.conv4(self.PoolUp4(y))

        concat = torch.cat([y,glob1,glob2,glob3,glob4],1)
        fully_connected = self.fc(concat)
        z = self.norm(fully_connected)

        return z



#Upsampling operation
class Up(nn.Module):

    def __init__(self, in_channels, out_channels, dilations, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DPR(in_channels, out_channels, dilations)

        else:
            raise Exception("Upscaling with other schemes rather than bilinear is not implemented")



    def forward(self, x1, x2):
        x1 = self.up(x1)
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


#Deformable convolution
#Used in DPR
class DeformableBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilate):
        super().__init__()

        self.offset = nn.Conv2d(in_channels, 2*kernel_size*kernel_size, kernel_size=3, padding = 1, dilation = 1)
        self.tan = nn.Hardtanh()
        self.deform = DeformConv2d(in_channels, out_channels, kernel_size = 3, stride = (1,1), padding = dilate, dilation = dilate)

    def forward(self,x):

        off = self.offset(x)
        off1 = self.tan(off)
        out = self.deform(x, off1)

        return out

#Deformable Pyramid Reception
class DPR(nn.Module):

    def __init__(self, in_channels, out_channels, dilations):
        super().__init__()

        self.conv0 = nn.Conv2d(in_channels, in_channels//4, kernel_size=3, padding=1)
        self.conv1 = DeformableBlock(in_channels, in_channels//8, kernel_size=3, dilate = dilations[0])
        self.conv2 = DeformableBlock(in_channels, in_channels//8, kernel_size=3, dilate = dilations[1])

        self.out = nn.Sequential(
                   nn.BatchNorm2d(in_channels//2),
                   nn.ReLU(inplace=True),
                   nn.Conv2d(in_channels//2, out_channels, kernel_size=3, padding=1),
                   nn.BatchNorm2d(out_channels),
                   nn.ReLU(inplace=True))


    def forward(self, x):

        x0 = self.conv0(x)
        x1 = self.conv1(x)
        x2 = self.conv2(x)

        y = torch.cat([x0, x1, x2], dim=1)
        y1 = self.out(y)

        return y1


#Convolution in Main architecture
class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

#Main Architecture
class DeepPyramid_VGG16(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(DeepPyramid_VGG16, self).__init__()

        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.Backbone = VGG16_Separate()

        self.PVF1 = PVF(input_channels=512, pool_sizes=[32, 9, 7, 5], dim=32)
        self.PVF2 = PVF(input_channels=256, pool_sizes=[64, 9, 7, 5], dim=64)
        self.PVF3 = PVF(input_channels=128, pool_sizes=[128, 9, 7, 5], dim=128)
        self.PVF4 = PVF(input_channels=64, pool_sizes=[256, 9, 7, 5], dim=256)

        self.up1 = Up(1024, 256, [3, 6, 7], bilinear)
        self.up2 = Up(512, 128, [3, 6, 7], bilinear)
        self.up3 = Up(256, 64, [3, 6, 7], bilinear)
        self.up4 = Up(128, 32, [3, 6, 7], bilinear)

        self.outc = OutConv(32, n_classes)




    def forward(self, x):

        out5, out4, out3, out2, out1 = self.Backbone(x)

        out1 = self.PVF1(out1)

        x1 = self.up1(out1, out2)
        x1 = self.PVF2(x1)

        x2 = self.up2(x1, out3)
        x2 = self.PVF3(x2)

        x3 = self.up3(x2, out4)
        x3 = self.PVF4(x3)

        x4 = self.up4(x3, out5)

        logits = self.outc(x4)


        return logits


# Training

In [7]:
#os.makedirs -> os.mkdir
def create_directory(new_dir):
    try:
        os.makedirs(new_dir)
        logging.info('Created checkpoint directory')
    except OSError:
        pass

def train_net(net,
              epochs=30,
              batch_size=1,
              lr=0.001,
              device='cuda', #cuda
              save_cp=True,
              size = (512,512)
              ):

    TESTS = []
    train_dataset = BasicDataset(dir_train_img, dir_train_mask, size = size)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=False)
    n_train = len(train_dataset)
    inference_step = np.floor(np.ceil(n_train/batch_size)/test_per_epoch)
    print(f'Inference Step:{inference_step}')

    test_dataset = BasicDataset(dir_test_img, dir_test_mask)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, pin_memory=False)#, drop_last=True)
    n_test = len(test_dataset)


    global_step = 0

    logging.info(f'''Starting training:
        Epochs:          {epochs}
        Batch size:      {batch_size}
        Learning rate:   {lr}
        Training size:   {n_train}
        Test size:       {n_test}
        Checkpoints:     {save_cp}
        Device:          {device.type}
    ''')

    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 2, gamma = 0.9)

    criterion = DiceBCELoss()
    test_counter = 1
    for epoch in range(epochs):
        net.train()


        epoch_loss = 0
        with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img') as pbar:
            for batch in train_loader:

                imgs = batch['image']
                true_masks = batch['mask']



                assert imgs.shape[1] == net.n_channels, \
                    f'Network has been defined with {net.n_channels} input channels, ' \
                    f'but loaded images have {imgs.shape[1]} channels. Please check that ' \
                    'the images are loaded correctly.'




                masks_pred = net(imgs)
                loss_main = criterion(masks_pred, true_masks)
                loss = loss_main
                epoch_loss += loss.item()


                pbar.set_postfix(**{'loss (batch)': loss.item()})

                optimizer.zero_grad()

                (loss_main).backward()
                nn.utils.clip_grad_value_(net.parameters(), 0.1)
                optimizer.step()

                pbar.update(imgs.shape[0])
                global_step += 1

                if (global_step) % (inference_step) == 0: # Should be changed if the condition that the n_train%batch_size != 0
                    for tag, value in net.named_parameters():
                        tag = tag.replace('.', '/')


                    val1, val2, val3, val4, val5, val6, val7, val8, inference_time = eval_dice_IoU_binary(net, test_loader, device, test_counter, save_test, save=False)

                    print(f'Validation Dice:{val1}')
                    print(f'Validation IoU:{val3}')
                    TESTS.append([val1, val2, val3, val4, val5, val6, val7, val8, inference_time, epoch_loss])

                    test_counter = test_counter+1


                    if net.n_classes > 1:
                         raise Exception("Not implemented for multi-class segmentation")
                    else:
                        logging.info('Validation Dice Coeff: {}'.format(val1))
                        logging.info('Validation IoU: {}'.format(val3))


        scheduler.step()

        if save_cp:
            if True:
                try:
                    os.makedirs(dir_checkpoint)
                    logging.info('Created checkpoint directory')
                except OSError:
                    pass
                torch.save(net.state_dict(),
                           dir_checkpoint + f'CP_epoch{epoch + 1}.pth')
                logging.info(f'Checkpoint {epoch + 1} saved !')

    val1, val2, val3, val4, val5, val6, val7, val8, inference_time = eval_dice_IoU_binary(net, test_loader, device, test_counter, save_test, save=True)
    save_metrics(TESTS, csv_name)



# Epochs and Paths

In [8]:
Net1 = DeepPyramid_VGG16
Categories = ['fold0']
Learning_Rates_init = [0.005]
epochs = 15
batch_size = 4
size = (512, 512)



Dataset_Path_Train='/content/drive/MyDrive/Capstone/Training Incision Segmentation/Knife Dataset/train imgs'
Dataset_Mask='/content/drive/MyDrive/Capstone/Training Incision Segmentation/Knife Dataset/train masks'
Dataset_Path_Test='/content/drive/MyDrive/Capstone/Training Incision Segmentation/Knife Dataset/val imgs'
Dataset_Mask_test='/content/drive/MyDrive/Capstone/Training Incision Segmentation/Knife Dataset/val masks'
mask_folder = '/content/drive/MyDrive/Capstone/Training Incision Segmentation/VGG16/output/masks/'
Results_path = '/content/drive/MyDrive/Capstone/Training Incision Segmentation/VGG16/output/results/'


# Main

In [ ]:
Visualization_path = 'visualization_DeepPyramid'
Checkpoint_path = 'checkpoints_DeepPyramid'
CSV_path = 'CSVs_DeepPyramid'
project_name = "DeepPyramidCataract"

load = False
load_path = ''
net_name = 'DeepPyramid_VGG16__'
test_per_epoch = 4
print("inside main")
print(f'Cuda Availability: {torch.cuda.is_available()}')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device : {device}')
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
logging.info(f'Using device {device}')


try:
    for c in range(len(Categories)):
        for LR in range(len(Learning_Rates_init)):
            print(f'Initializing the learning rate: {Learning_Rates_init[LR]}')

            dir_train_img = Dataset_Path_Train
            dir_train_mask = Dataset_Mask
            dir_test_img = Dataset_Path_Test
            dir_test_mask = Dataset_Mask_test
            save_test = Results_path + Visualization_path +'CATARACT_'+net_name +str(Learning_Rates_init[LR])+'_'+str(Categories[c])+'/'
            dir_checkpoint = Results_path + Checkpoint_path +'CATARACT_'+ net_name +str(Learning_Rates_init[LR])+'_'+str(Categories[c])+'/'
            csv_name = Results_path + CSV_path +'CATARACT_'+net_name +str(Learning_Rates_init[LR])+'_'+str(Categories[c])+'.csv'
            create_directory(Results_path + Visualization_path)
            create_directory(Results_path + Checkpoint_path)
            create_directory(Results_path + CSV_path)

            #Model Calling
            net = Net1(n_classes=1, n_channels=3, bilinear=True)
            logging.info(f'Network:\n'
                         f'\t{net.n_channels} input channels\n'
                         f'\t{net.n_classes} output channels (classes)\n')

            #Loading Model to GPU
            if load:
                net.load_state_dict(
                    torch.load(load_path, map_location=device)
                )
                logging.info(f'Model loaded from {str(load_path)}')

            net.to(device=device)
            train_net(net=net,
                      epochs=epochs,
                      batch_size=batch_size,
                      lr=Learning_Rates_init[LR],
                      device=device,
                      size = size)

except KeyboardInterrupt:
    torch.save(net.state_dict(), 'INTERRUPTED.pth')
    logging.info('Saved interrupt')
    try:
        sys.exit(0)
    except SystemExit:
        os._exit(0)


inside main
Cuda Availability: True
device : cuda
Initializing the learning rate: 0.005


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 208MB/s]


Here inside dataset
Inference Step:12.0
Here inside dataset


Epoch 1/15:   0%|          | 0/192 [00:07<?, ?img/s, loss (batch)=1.23]

Train Dice: tensor([0.0336, 0.0386, 0.0387, 0.0646], device='cuda:0',
       grad_fn=<DivBackward0>)
Train BCE: 0.7578102946281433


...

Train Dice: tensor([0.9710, 0.9686, 0.9754, 0.9748], device='cuda:0',
       grad_fn=<DivBackward0>)
Train BCE: 0.0018559731543064117


Epoch 15/15: 100%|██████████| 192/192 [02:23<00:00,  1.47img/s, loss (batch)=0.00707]

Train Dice: tensor([0.9713, 0.9729, 0.9752, 0.9742], device='cuda:0',
       grad_fn=<DivBackward0>)
Train BCE: 0.0020968937315046787



Epoch 15/15: 100%|██████████| 192/192 [02:30<00:00,  1.27img/s, loss (batch)=0.00707]


Validation Dice:0.8700288571417332
Validation IoU:0.7798646204173565
